In [37]:
import pandas as pd
from transformers import AutoTokenizer
import torch
import torch
from transformers import AutoModelForMaskedLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from tqdm import tqdm

In [65]:
csv_path = '/home/uddeshya.singh/DataSet_triplet/combined_cleaned_triplet_train_PUNJAB.csv'
df = pd.read_csv(csv_path)

In [66]:
addresses = df['cleaned_address'].tolist()

In [57]:
df.head()

city  pincode                                    cleaned_address  \
0  Moga District   142048                      00 baghela wala Moga District   
1  Moga District   142048   01 daroli bhai near safu walaa road Moga Dist...   
2           Moga   142048               02 isf college of pharmacy moga Moga   
3           Moga   142048           1 dhalle_ke br canda zira road moga Moga   
4  Moga District   142048        1 ghal kalan moga subdistrict Moga District   

   lookup_lat  lookup_lng  no_of_delivereies  
0   30.807182   74.997443                  6  
1   30.807944   75.047640                  4  
2   30.827942   75.118181                  6  
3   30.850509   75.137326                  7  
4   30.818855   75.108905                  4

In [58]:
tokenizer_path = '/home/devanapalli.ravi/disk_1/paper/data/untrained_models/distilroberta-base/'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

In [83]:
class AddressDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

def batch_tokenize(addresses, tokenizer, batch_size=100, max_length=512):
    encodings = {'input_ids': [], 'attention_mask': []}
    for i in tqdm(range(0, len(addresses), batch_size), desc="Tokenizing", unit="batch"):
        batch = addresses[i:i+batch_size]
        batch_encodings = tokenizer(batch, truncation=True, padding=True, max_length=max_length)
        encodings['input_ids'].extend(batch_encodings['input_ids'])
        encodings['attention_mask'].extend(batch_encodings['attention_mask'])
    return encodings

In [84]:
encodings = batch_tokenize(addresses, tokenizer)

Tokenizing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 233.32batch/s]


In [85]:
dataset = AddressDataset(encodings)

In [86]:
type(encodings)

dict

In [89]:
len(encodings['input_ids'][2])

35

In [47]:
model = AutoModelForMaskedLM.from_pretrained(tokenizer_path)

Some weights of the model checkpoint at /home/devanapalli.ravi/disk_1/paper/data/untrained_models/distilroberta-base/ were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [48]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)


In [49]:
training_args = TrainingArguments(
    output_dir='./results',           # output directory
    overwrite_output_dir=True,        # overwrite the content of the output directory
    num_train_epochs=1,               # number of training epochs
    per_device_train_batch_size=16,   # batch size for training
    save_steps=10_000,                # after how many steps to save a checkpoint
    save_total_limit=2,               # limit the total amount of checkpoints
    logging_dir='./logs',             # directory for storing logs
    logging_steps=200,                # log saving step
)

trainer = Trainer(
    model=model,                       # the instantiated 🤗 Transformers model to be trained
    args=training_args,                # training arguments, defined above
    data_collator=data_collator,       # data collator
    train_dataset=dataset,             # training dataset
)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [50]:
trainer.train()

/home/uddeshya.singh/miniconda3/envs/uddeshya_env/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=16, training_loss=3.9621472358703613, metrics={'train_runtime': 5.1671, 'train_samples_per_second': 193.531, 'train_steps_per_second': 3.096, 'total_flos': 13469402232000.0, 'train_loss': 3.9621472358703613, 'epoch': 1.0})